In [1]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import re
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import f1_score, accuracy_score

In [27]:
train_data=pd.read_csv(r"C:\Users\user\Desktop\Data sets\train_senti.csv")
test_data=pd.read_csv(r"C:\Users\user\Desktop\Data sets\test_senti.csv")
submission_data=pd.read_csv(r"C:\Users\user\Desktop\Data sets\submission_seti.csv")

In [28]:
train_data.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [29]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      31962 non-null  int64 
 1   label   31962 non-null  int64 
 2   tweet   31962 non-null  object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


In [30]:
train_data['label'].value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [31]:
train_data['tweet'][76]

'@user wrapping up #senseaboutmaths @user 6th   @user @user @user '

In [32]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [33]:
stop_words = set(stopwords.words('english'))
stop = [x.lower() for x in stop_words]
lemma = WordNetLemmatizer()

shortcuts = {'u': 'you', 'y': 'why', 'r': 'are', 'doin': 'doing', 'hw': 'how', 'k': 'okay', 'm': 'am', 'b4': 'before',
            'idc': "i do not care", 'ty': 'thankyou', 'wlcm': 'welcome', 'bc': 'because', '<3': 'love', 'xoxo': 'love',
            'ttyl': 'talk to you later', 'gr8': 'great', 'bday': 'birthday', 'awsm': 'awesome', 'gud': 'good', 'h8': 'hate',
            'lv': 'love', 'dm': 'direct message', 'rt': 'retweet', 'wtf': 'hate', 'idgaf': 'hate',
             'irl': 'in real life', 'yolo': 'you only live once'}

def clean(text):
    text = text.lower()
    # keep alphanumeric characters only
    text = re.sub('\W+', ' ', text).strip()
    text = text.replace('user', '')
    # tokenize
    text_token = word_tokenize(text)
    # replace shortcuts using dict
    full_words = []
    for token in text_token:
        if token in shortcuts.keys():
            token = shortcuts[token]
        full_words.append(token)
#     text = " ".join(full_words)
#     text_token = word_tokenize(text)
    # stopwords removal
#     words = [word for word in full_words if word not in stop]
    words_alpha = [re.sub(r'\d+', '', word) for word in full_words]
    words_big = [word for word in words_alpha if len(word)>2]
    stemmed_words = [lemma.lemmatize(word) for word in words_big]
    # join list elements to string
    clean_text = " ".join(stemmed_words)
    clean_text = clean_text.replace('   ', ' ')
    clean_text = clean_text.replace('  ', ' ')
    return clean_text

In [34]:
hypocrite = []
for i in range(len(train_data['tweet'])):
    if 'hypocrite' in train_data['tweet'][i]:
        if train_data['label'][i] == 1:
            hypocrite.append('racist')
        else:
            hypocrite.append('good')
    else:
        hypocrite.append('good')
df = pd.DataFrame(columns=['hypocrite'], data=hypocrite)
print(df['hypocrite'].value_counts())

train_data['hypocrite'] = hypocrite


good      31957
racist        5
Name: hypocrite, dtype: int64


In [35]:
train_data['combined'] = train_data['tweet'].apply(str) + ' ' + train_data['hypocrite'].apply(str)

In [36]:
train_data.head()

,id,label,tweet,hypocrite,combined
0,1,0,@user when a father is dysfunctional and is s...,good,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...,good,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty,good,bihday your majesty good
3,4,0,#model i love u take with u all the time in ...,good,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation,good,factsguide: society now #motivation good


In [37]:
train_data['combined'][0]

' @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run good'

In [38]:
X_train = train_data.combined
y = train_data.label
X_test = test_data.tweet

In [39]:
X_train

0         @user when a father is dysfunctional and is s...
1        @user @user thanks for #lyft credit i can't us...
2                                 bihday your majesty good
3        #model   i love u take with u all the time in ...
4              factsguide: society now    #motivation good
                               ...                        
31957    ate @user isz that youuu?ðððððð...
31958      to see nina turner on the airwaves trying to...
31959    listening to sad songs on a monday morning otw...
31960    @user #sikh #temple vandalised in in #calgary,...
31961                thank you @user for you follow   good
Name: combined, Length: 31962, dtype: object

In [40]:
X_test

0        #studiolife #aislife #requires #passion #dedic...
1         @user #white #supremacists want everyone to s...
2        safe ways to heal your #acne!!    #altwaystohe...
3        is the hp and the cursed child book up for res...
4          3rd #bihday to my amazing, hilarious #nephew...
                               ...                        
17192    thought factory: left-right polarisation! #tru...
17193    feeling like a mermaid ð #hairflip #neverre...
17194    #hillary #campaigned today in #ohio((omg)) &am...
17195    happy, at work conference: right mindset leads...
17196    my   song "so glad" free download!  #shoegaze ...
Name: tweet, Length: 17197, dtype: object

In [41]:
y

0        0
1        0
2        0
3        0
4        0
        ..
31957    0
31958    0
31959    0
31960    1
31961    0
Name: label, Length: 31962, dtype: int64

In [42]:
clean_Xtest = X_test.apply(lambda x: clean(x))

In [43]:
clean_Xtest = X_test.apply(lambda x: clean(x))

In [44]:
print(len(X_train))
print(len(clean_Xtest))
print(len(y))

31962
17197
31962


In [45]:
vectorizer = CountVectorizer(max_df=0.5)
# vectorizer = TfidfVectorizer(ngram_range=(1,3), max_df=0.5)

X = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(clean_Xtest)

In [46]:
model = LinearSVC(penalty='l2', C=0.5, dual=False, random_state=0, max_iter=1000)
print(model)

LinearSVC(C=0.5, dual=False, random_state=0)


In [47]:
# split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.30, random_state=15)

# calculate f1 score
model.fit(X_train,y_train)
y_pred = model.predict(X_val)
print('Accuracy:', accuracy_score(y_pred, y_val))
print("F1 Score: ", f1_score(y_pred, y_val))

Accuracy: 0.960892689540098
F1 Score:  0.6770025839793282


In [48]:
df = pd.DataFrame()
df['y_pred'] = y_pred
df['y_pred'].value_counts()

0    9118
1     471
Name: y_pred, dtype: int64

In [49]:
# train model with full data and predict for new samples
model.fit(X, y)
y_pred = model.predict(X_test)

In [51]:
# save it to submission csv
submission_data['label'] = y_pred
submission_data.to_csv('senti.csv', index=False)